In [8]:
from streamlit.connections import ExperimentalBaseConnection
import os
import requests
import streamlit as st
import json
import plotly.graph_objects as go
from datetime import datetime



In [9]:
session = requests.Session()

In [10]:
countries_1 = session.get(f'https://api.openaq.org/v2/countries?limit=100&page=1&sort=asc&order_by=name')
countries_2 = session.get(f'https://api.openaq.org/v2/countries?limit=100&page=2&sort=asc&order_by=name')

In [11]:
countries = countries_1.json()["results"] + countries_2.json()["results"]

In [12]:
transformed_dict = {country['name']: country['code'] for country in countries}
transformed_dict

{'Afghanistan': 'AF',
 'Algeria': 'DZ',
 'Angola': 'AO',
 'Antarctica': 'AQ',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Bangladesh': 'BD',
 'Belgium': 'BE',
 'Belize': 'BZ',
 'Benin': 'BJ',
 'Bolivia': 'BO',
 'Bosnia and Herz.': 'BA',
 'Brazil': 'BR',
 'Bulgaria': 'BG',
 'Burkina Faso': 'BF',
 'Burundi': 'BI',
 'Cameroon': 'CM',
 'Canada': 'CA',
 'Central African Rep.': 'CF',
 'Chad': 'TD',
 'Chile': 'CL',
 'China': 'CN',
 'Colombia': 'CO',
 'Costa Rica': 'CR',
 "Côte d'Ivoire": 'CI',
 'Croatia': 'HR',
 'Cyprus': 'CY',
 'Czechia': 'CZ',
 'Dem. Rep. Congo': 'CD',
 'Denmark': 'DK',
 'Dominican Rep.': 'DO',
 'Ecuador': 'EC',
 'Egypt': 'EG',
 'El Salvador': 'SV',
 'Estonia': 'EE',
 'Ethiopia': 'ET',
 'Fiji': 'FJ',
 'Finland': 'FI',
 'France': 'FR',
 'Gabon': 'GA',
 'Gambia': 'GM',
 'Georgia': 'GE',
 'Germany': 'DE',
 'Ghana': 'GH',
 'Greece': 'GR',
 'Greenland': 'GL',
 'Guatemala': 'GT',
 'Guinea': 'GN',
 'Guyana': 'GY',
 'Honduras':

In [71]:
transformed_dict["Afghanistan"]

'AF'

In [ ]:
countries

In [4]:
code="GR"

In [13]:
len(transformed_dict)

141

In [1]:
params = {"limit": 1000, "page": 1, "offset":0, "sort": "desc", "radius":1000, "order_by": "lastUpdated", "dumpRaw":"false"}
params["country_id"] = "GR"
response = session.get(
                    'https://api.openaq.org/v2/locations',
                    params=params
                )

NameError: name 'session' is not defined

In [43]:
response = session.get(f'https://api.openaq.org/v2/locations?limit=1000&page=1&offset=0&sort=desc&radius=1000&country_id={code}&order_by=lastUpdated&dumpRaw=false')

In [44]:
test = response.json()

In [45]:
def is_daytime():
    # Get the current local time
    now = datetime.now()
    # Set the hours during which it's considered daytime (e.g., 6 AM to 6 PM)
    daytime_start = now.replace(hour=6, minute=0, second=0)
    daytime_end = now.replace(hour=18, minute=0, second=0)
    # Check if the current time is within the daytime range
    return daytime_start <= now <= daytime_end

In [61]:
def visualize_variable_on_map(data_dict, variable):
    is_day = is_daytime()
    mapbox_style = 'carto-darkmatter' if not is_day else 'open-street-map'

    # Initialize lists to store data for multiple locations
    latitudes = []
    longitudes = []
    values = []
    display_names = []
    last_updated = []

    custom_markers = {
        'humidity': "&#x1F4A7;",  # Humidity - Raindrop Emoji
        'others': "&#x25B2;",  
    }

    # Loop through the results and extract relevant data for each location
    for result in data_dict.get('results', []):
        measurements = result.get('parameters', [])
        for measurement in measurements:
            if measurement['parameter'] == variable:
                value = measurement['lastValue']
                display_name = measurement['displayName']
                latitude = result['coordinates']['latitude']
                longitude = result['coordinates']['longitude']
                last_updated_value = result['lastUpdated']

                latitudes.append(latitude)
                longitudes.append(longitude)
                values.append(value)
                display_names.append(display_name)
                last_updated.append(last_updated_value)

    if not latitudes or not longitudes or not values:
        print(f"{variable} data not found.")
        return

    # Create the visualization
    fig = go.Figure()

    marker = [custom_markers["humidity"] if variable=="humidity" else custom_markers["others"]]

    # Add a single scatter mapbox trace with all locations
    fig.add_trace(go.Scattermapbox(
        lat=latitudes,
        lon=longitudes,
        mode='markers+text',
        marker=dict(
            size=20,
            color=values,
            colorscale='Viridis',  # You can choose other color scales as well
            colorbar=dict(title=f'{variable.capitalize()}')
        ),
        text=[f"{marker[0]} {display_name}: {values[i]}<br>Last Updated: {last_updated[i]}" for i, display_name in enumerate(display_names)],
        hoverinfo='text',
    ))
    
    # Update map layout
    fig.update_layout(
        mapbox=dict(
            style=mapbox_style,  # Choose the desired map style
            zoom=5,  # Adjust the initial zoom level as needed
            center=dict(lat=sum(latitudes) / len(latitudes), lon=sum(longitudes) / len(longitudes))
        ),
        margin=dict(l=0, r=0, t=0, b=0)
    )

    # Show the map
    fig.show()

visualize_variable_on_map(test, "co")


In [ ]:
test["results"]

In [56]:
params = []
parameters = test["results"][1]["parameters"]

for parameter in parameters:
    p = parameter["parameter"]
    if p not in params:
        params.append(p)

In [59]:
params = []
for i in range(len(test["results"])):
    parameters = test["results"][i]["parameters"]

    for parameter in parameters:
        p = parameter["parameter"]
        if p not in params:
            params.append(p)

In [60]:
params

['um025',
 'humidity',
 'pressure',
 'um050',
 'temperature',
 'um003',
 'um005',
 'pm1',
 'pm10',
 'um100',
 'pm25',
 'um010',
 'voc',
 'pm4',
 'no',
 'no2',
 'o3',
 'so2',
 'co']

In [ ]:
test["results"][0]